# Exploring data

Retrieving datasets, filtering, pivoting and merging

In [1]:
import pandas as pd

## Merging happiness datasets from years 2015-2019

Since the different csv for the different years have different column names, made a dictionary to connect the correct ones

In [2]:
column_dict = {"Happiness.Rank" : "Happiness Rank",
               "Overall rank" : "Happiness Rank",
               "Country or region" : "Country",
               "Happiness.Score" : "Happiness Score",
               "Score" : "Happiness Score",
               "Economy (GDP per Capita)" : "GDP",
               "Economy..GDP.per.Capita." : "GDP",
               "GDP per capita" : "GDP",
               "Health (Life Expectancy)" : "Life expectancy",
               "Health..Life.Expectancy." : "Life expectancy",
               "Healthy life expectancy" : "Life expectancy",
               "Trust (Government Corruption)" : "Corruption",
               "Trust..Government.Corruption." : "Corruption",
               "Perceptions of corruption" : "Corruption"
              }

Then, retrieve all 5 datasets and concat them in to one `happiness` dataframe

In [3]:
years = range(2015, 2020)

ind_years = []
columns_to_keep = ["Country", "Year", "Happiness Rank", "Happiness Score", "GDP", "Life expectancy", "Corruption"]

for year in years:
    path = "../data/worldhappiness/" + str(year) + ".csv"
    df = pd.read_csv(path)
    df.rename(columns=column_dict, inplace=True)
    df["Year"] = year
    df = df[columns_to_keep]
    ind_years.append(df)
    
happiness = pd.concat(ind_years, ignore_index=True)

And that's all we need to do for now with the happiness data

In [4]:
happiness.head() 

,Country,Year,Happiness Rank,Happiness Score,GDP,Life expectancy,Corruption
0,Switzerland,2015,1,7.587,1.39651,0.94143,0.41978
1,Iceland,2015,2,7.561,1.30232,0.94784,0.14145
2,Denmark,2015,3,7.527,1.32548,0.87464,0.48357
3,Norway,2015,4,7.522,1.45900,0.88521,0.36503
4,Canada,2015,5,7.427,1.32629,0.90563,0.32957


## Preparing SGD Data

First, read in the excel sheet for the SGD data

In [5]:
sgd_df = pd.read_excel("../data/sdgindicators/data.xlsx")
sgd_df.head()

,setting,date,source,indicator_abbr,indicator_name,dimension,subgroup,estimate,se,ci_lb,...,iso3,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,whoreg6,wbincome2023,dataset_id,update
0,Afghanistan,2000,UN SDG Indicators Database,SI_POV_EMP1,1.1.1 Employed population below international ...,Age (2 groups) (15-25+),15-24 years,66.300003,NaN,NaN,...,AFG,0,100,0,0,0,Eastern Mediterranean,Low income,rep_sdg,24 March 2023
1,Afghanistan,2000,UN SDG Indicators Database,SI_POV_EMP1,1.1.1 Employed population below international ...,Age (2 groups) (15-25+),25+ years,66.300003,NaN,NaN,...,AFG,0,100,0,0,1,Eastern Mediterranean,Low income,rep_sdg,24 March 2023
2,Afghanistan,2000,UN SDG Indicators Database,SI_POV_EMP1,1.1.1 Employed population below international ...,Sex,Female,71.599998,NaN,NaN,...,AFG,0,100,0,0,0,Eastern Mediterranean,Low income,rep_sdg,24 March 2023
3,Afghanistan,2000,UN SDG Indicators Database,SI_POV_EMP1,1.1.1 Employed population below international ...,Sex,Male,65.400002,NaN,NaN,...,AFG,0,100,0,0,1,Eastern Mediterranean,Low income,rep_sdg,24 March 2023
4,Afghanistan,2001,UN SDG Indicators Database,SI_POV_EMP1,1.1.1 Employed population below international ...,Age (2 groups) (15-25+),15-24 years,66.900002,NaN,NaN,...,AFG,0,100,0,0,0,Eastern Mediterranean,Low income,rep_sdg,24 March 2023


Then we check the column names and make a list for the indicators we're interested in

In [6]:
sgd_df.columns

Index(['setting', 'date', 'source', 'indicator_abbr', 'indicator_name',
       'dimension', 'subgroup', 'estimate', 'se', 'ci_lb', 'ci_ub',
       'population', 'flag', 'setting_average', 'iso3', 'favourable_indicator',
       'indicator_scale', 'ordered_dimension', 'subgroup_order',
       'reference_subgroup', 'whoreg6', 'wbincome2023', 'dataset_id',
       'update'],
      dtype='object')

In [7]:
interesting_indicators = [
    "1.1.1 Employed population below international poverty line (%)",
    "1.1.1 Employed population below international poverty line (%) - Female",
    "1.1.1 Employed population below international poverty line (%) - Male",
    "1.1.1 Population below international poverty line (%)",
    '3.4.2 Suicide mortality rate (deaths per 100 000 population)',
    '4.1.2 Completion rate (%)',
    '4.1.2 Completion rate (%) - Lower secondary education',
    '4.1.2 Completion rate (%) - Primary education',
    '4.1.2 Completion rate (%) - Upper secondary education',
    '8.5.2 Unemployment rate (%)',
    '8.5.2 Unemployment rate (%) - Female',
    '8.5.2 Unemployment rate (%) - Male',
]

columns_keep = [
    'setting', 
    'date',
    'indicator_name',
    'dimension', 
    'subgroup', 
    'estimate',
    'setting_average',
    'indicator_scale'
]

Then we filter for the years 2015-2019, since that matches our happiness dataset, and filter only the indicators and columns we want

In [8]:
# filter out years and columns of interest
sgd = sgd_df[(sgd_df["date"] >= 2015) & 
             (sgd_df["date"] <= 2019) & 
             (sgd_df["indicator_name"].isin(interesting_indicators))][columns_keep]

Since in the `estimate` column some values are from 0-100 and some from 0-10000, we normalize values depending on what is indicated in the `indicator_scale` column. Furthermore, rename some columns for better understandability

In [9]:
# normalize estimate based on indicator scale
sgd['value'] = sgd.apply(lambda row: (row['estimate'] / row['indicator_scale']) * 100, axis=1)
sgd['average'] = sgd.apply(lambda row: (row['setting_average'] / row['indicator_scale']) * 100, axis=1)
# drop non-normalizd columns
sgd_norm = sgd.drop(columns=["estimate", "setting_average", "indicator_scale", "dimension"])
# rename columns for convenience
sgd_norm.rename(columns={"setting" : "Country", 
                         "date" : "Year",
                         "indicator_name" : "Data"},
                inplace=True)
# display new head
sgd_norm.head()

,Country,Year,Data,subgroup,value,average
60,Afghanistan,2015,1.1.1 Employed population below international ...,15-24 years,46.000000,43.700001
61,Afghanistan,2015,1.1.1 Employed population below international ...,25+ years,42.799999,43.700001
62,Afghanistan,2015,1.1.1 Employed population below international ...,Female,50.700001,43.700001
63,Afghanistan,2015,1.1.1 Employed population below international ...,Male,42.099998,43.700001
64,Afghanistan,2016,1.1.1 Employed population below international ...,15-24 years,45.099998,42.700001


Now, the average value is stored in a different column, but we want to move that into a distinct subgroup and also store it into the `value` column. We create a new dataframe with one row for each group of data, and then concat that to the original, resulting in a cleaner dataframe

In [10]:
# Create a new DataFrame to store the results
average_sgd = pd.DataFrame()

# Create new rows with average stores in "value" column
for group, group_df in sgd_norm.groupby(['Country', 'Year', 'Data']):
    if 'average' not in group_df['subgroup'].values:
        new_row = group_df.iloc[0].copy()
        new_row['subgroup'] = 'average'
        new_row['value'] = new_row['average']
        average_sgd = pd.concat([average_sgd, new_row.to_frame().T], ignore_index=True)

# Concatenate with original and sort
sgd_w_average = pd.concat([sgd_norm, average_sgd], ignore_index=True)
sgd_w_average = sgd_w_average.sort_values(by=['Country', 'Year', 'Data', 'subgroup']).reset_index(drop=True)

# Drop average column
sgd_w_average.drop(columns="average", inplace=True)

sgd_w_average.head()

,Country,Year,Data,subgroup,value
0,Afghanistan,2015,1.1.1 Employed population below international ...,15-24 years,46.0
1,Afghanistan,2015,1.1.1 Employed population below international ...,25+ years,42.799999
2,Afghanistan,2015,1.1.1 Employed population below international ...,Female,50.700001
3,Afghanistan,2015,1.1.1 Employed population below international ...,Male,42.099998
4,Afghanistan,2015,1.1.1 Employed population below international ...,average,43.700001


Now we want to move the data description from the "Data" column, and pivot the table so that we have a column for each data type

In [11]:
pivot_df = sgd_w_average.pivot_table(
    index=['Country', 'Year'],
    columns=['Data', 'subgroup'],
    values=['value'],
    aggfunc='first'  # Use 'first' to handle duplicate entries, if any
).reset_index()

pivot_df.head()

Country  Year  \
Data                          
subgroup                      
0         Afghanistan  2015   
1         Afghanistan  2016   
2         Afghanistan  2017   
3         Afghanistan  2018   
4         Afghanistan  2019   

                                                                  value  \
Data     1.1.1 Employed population below international poverty line (%)   
subgroup                                                    15-24 years   
0                                                 46.000000               
1                                                 45.099998               
2                                                 43.900002               
3                                                 43.400002               
4                                                 42.700001               

                                                      \
Data                                                   
subgroup  25+ years     Female       Male    average   
0         42.799999  50.700001  42.099998  43.700001   
1         41.700001  49.500000  41.000000  42.700001   
2         40.200001  48.000000  39.500000  41.299999   
3         39.400002  47.400002  38.799999  40.599998   
4         38.599998  46.599998  38.000000  39.799999   

                                                                                  \
Data     1.1.1 Employed population below international poverty line (%) - Female   
subgroup                                                             15-24 years   
0                                                 50.500000                        
1                                                 49.500000                        
2                                                 48.099998                        
3                                                 47.500000                        
4                                                 46.799999                        

                                ...                                       \
Data                            ... 8.5.2 Unemployment rate (%) - Female   
subgroup  25+ years    average  ...                          15-24 years   
0         50.799999  50.700001  ...                                  NaN   
1         49.599998  49.500000  ...                                  NaN   
2         48.000000  48.000000  ...                                 21.4   
3         47.299999  47.400002  ...                                  NaN   
4         46.500000  46.599998  ...                                  NaN   

                                                                               \
Data                                                                            
subgroup 25+ years Persons with disability Persons without disability average   
0              NaN                     NaN                        NaN     NaN   
1              NaN                     NaN                        NaN     NaN   
2             10.1                    13.8                       14.0    14.0   
3              NaN                     NaN                        NaN     NaN   
4              NaN                     NaN                        NaN     NaN   

                                                                               \
Data     8.5.2 Unemployment rate (%) - Male                                     
subgroup                        15-24 years 25+ years Persons with disability   
0                                       NaN       NaN                     NaN   
1                                       NaN       NaN                     NaN   
2                                 16.299999       7.9                    12.4   
3                                       NaN       NaN                     NaN   
4                                       NaN       NaN                     NaN   

                                             
Data                                         
subgroup Persons without disability average

# Merging both Datasets

Firstly, we check if the countries roughly align

In [12]:
hap_countries = set(happiness["Country"])
sgd_countries = set(pivot_df["Country"])
print(len(hap_countries))
print(len(sgd_countries))

len(hap_countries.intersection(sgd_countries))
print(hap_countries.difference(sgd_countries), "\n")
print(sgd_countries.difference(hap_countries))

170
190
{'Macedonia', 'Iran', 'Taiwan', 'Ivory Coast', 'United States', 'Laos', 'Palestinian Territories', 'United Kingdom', 'Russia', 'Somaliland Region', 'Northern Cyprus', 'Trinidad & Tobago', 'Somaliland region', 'Swaziland', 'Hong Kong', 'Czech Republic', 'Bolivia', 'Netherlands', 'Hong Kong S.A.R., China', 'Taiwan Province of China', 'Vietnam', 'Tanzania', 'North Cyprus', 'Congo (Brazzaville)', 'Moldova', 'Congo (Kinshasa)', 'Turkey', 'South Korea', 'Syria', 'Venezuela', 'Kosovo'} 

{'Maldives', 'Micronesia (Federated States of)', 'Vanuatu', 'Viet Nam', 'Cook Islands', 'Bahamas', 'Timor-Leste', 'United Republic of Tanzania', 'Guyana', 'Cabo Verde', 'Equatorial Guinea', 'Brunei Darussalam', 'Czechia', 'Seychelles', 'Congo', 'Fiji', 'United States of America', 'The United Kingdom', "Democratic People's Republic of Korea", 'Tonga', 'Iran (Islamic Republic of)', 'Papua New Guinea', 'Tuvalu', 'Sao Tome and Principe', 'Saint Lucia', 'Barbados', "Lao People's Democratic Republic", 'Kiri

We can fix some by renaming the countries in the sgd dataset

In [13]:
sgd_country_rename = {
    'Eswatini':'Swaziland',
    "Lao People's Democratic Republic":"Laos",
    'United States of America':'United States',
    'The United Kingdom':'United Kingdom',
    'Czechia':'Czech Republic',
    'Netherlands (Kingdom of the)':'Netherlands',
    "Viet nam" : "Vietnam",
    'Russian Federation' : 'Russia',
    "Republic of Moldova" : 'Moldova',
    'Bolivia (Plurinational State of)':'Bolivia',
    'Republic of Korea': 'South Korea',
    'Iran (Islamic Republic of)': 'Iran',
    'Syrian Arab Republic': 'Syria',
    'Türkiye':'Turkey',
    'United Republic of Tanzania': 'Tanzania',
    'occupied Palestinian territory':'Palestinian Territories',
    'Congo':'Congo (Brazzaville)',
    'Democratic Republic of the Congo':'Congo (Kinshasa)',
    'Venezuela (Bolivarian Republic of)':'Venezuela',
    
}

In [14]:
# replace names in SGD dataset
pivot_df['Country'] = pivot_df['Country'].replace(sgd_country_rename)

hap_countries = set(happiness["Country"])
sgd_countries = set(pivot_df["Country"])
print(len(hap_countries))
print(len(sgd_countries))
print(len(hap_countries.intersection(sgd_countries)))

170
190
157


So there are `157` countries that are present in both datasets

Now, we can merge the datasets on year and country

In [15]:
combined = pd.merge(happiness, pivot_df, on=['Country', 'Year'], how="inner").sort_values(by=['Country', 'Year']).reset_index(drop=True)
print(combined.iloc[9:15])

    Country  Year  Happiness Rank  Happiness Score       GDP  Life expectancy  \
9   Albania  2019             107            4.719  0.947000         0.874000   
10  Algeria  2015              68            5.605  0.939290         0.617660   
11  Algeria  2016              38            6.355  1.052660         0.618040   
12  Algeria  2017              53            5.872  1.091864         0.617585   
13  Algeria  2018              84            5.295  0.979000         0.687000   
14  Algeria  2019              88            5.211  1.002000         0.785000   

    Corruption  \
9     0.027000   
10    0.173830   
11    0.161570   
12    0.146096   
13    0.135000   
14    0.114000   

    (value, 1.1.1 Employed population below international poverty line (%), 15-24 years)  \
9                                                 0.8                                      
10                                                0.1                                      
11                           

/var/folders/jt/d69hwrb156jbgydttvzhcngc0000gn/T/ipykernel_20555/2688176290.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 3 on the right)
  combined = pd.merge(happiness, pivot_df, on=['Country', 'Year'], how="inner").sort_values(by=['Country', 'Year']).reset_index(drop=True)
/var/folders/jt/d69hwrb156jbgydttvzhcngc0000gn/T/ipykernel_20555/2688176290.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  combined = pd.merge(happiness, pivot_df, on=['Country', 'Year'], how="inner").sort_values(by=['Country', 'Year']).reset_index(drop=True)


And we get the full dataset, which we will merge to pickle for easy use.

To read it back later, use `df = pd.read_pickle('../data/combined_df.pkl')`

In [16]:
import pickle

combined.to_pickle('../data/combined_df.pkl')

In [44]:
df = pd.read_pickle('../data/combined_df.pkl')
# df['Country'].unique()
# df.columns

There is a lot of missing data for each year in most of the countries so we decided to drop some of those countries and keep the ones that contain most of the data.
Later on we could also go back to some of the countries, some of them are countries that are labaled as underdeveloped so the lack of information on indicators such as `4.1.2 Completion rate (%)` or `8.5.2 Unemployment rate (%) - Male, Persons with disability` is understandable. This knowledge could also explain the `happiness rating` that has been assigned to those countries. 

In [20]:
columns_to_check_nan = df.columns[7:]

for country in df['Country'].unique():
    nan_data = df.loc[df['Country'] == country, columns_to_check_nan].isnull().sum()
    nan_columns = nan_data[nan_data > 0].index
    
    if not nan_columns.empty:
        print(f"Country: {country}")
        for column in nan_columns:
            nan_years = df.loc[(df['Country'] == country) & df[column].isnull(), 'Year'].tolist()
            nan_count = nan_data[column]
            print(f"  - Column with NaN: {column}")
            print(f"    - Years with NaN: {nan_years}")
            print(f"    - Number of NaN values: {nan_count}")

Country: Afghanistan
  - Column with NaN: ('value', '1.1.1 Population below international poverty line (%)', '15-64 years')
    - Years with NaN: [2015, 2016, 2017, 2018, 2019]
    - Number of NaN values: 5
  - Column with NaN: ('value', '1.1.1 Population below international poverty line (%)', '65+ years')
    - Years with NaN: [2015, 2016, 2017, 2018, 2019]
    - Number of NaN values: 5
  - Column with NaN: ('value', '1.1.1 Population below international poverty line (%)', '<15 years')
    - Years with NaN: [2015, 2016, 2017, 2018, 2019]
    - Number of NaN values: 5
  - Column with NaN: ('value', '1.1.1 Population below international poverty line (%)', 'Female')
    - Years with NaN: [2015, 2016, 2017, 2018, 2019]
    - Number of NaN values: 5
  - Column with NaN: ('value', '1.1.1 Population below international poverty line (%)', 'Male')
    - Years with NaN: [2015, 2016, 2017, 2018, 2019]
    - Number of NaN values: 5
  - Column with NaN: ('value', '1.1.1 Population below internatio

In [45]:
employed_poverty_rate = df.filter(like='Employed').columns
#employed_poverty_rate.loc[:, 'Country'] = df['Country'].copy()
#employed_poverty_rate.loc[:, 'Year'] = df['Year'].copy()
#employed_poverty_rate.isnull().sum()

df.drop(columns=employed_poverty_rate, inplace=True)

In [65]:
poverty_columns = df.filter(like='1.1.1').columns
# suicide_columns = df.columns[df.columns.str.contains('3.4.2')]
# completion_columns = df.columns[df.columns.str.contains('4.1.2')]
# unemployment_columns = df.columns[df.columns.str.contains('8.5.2')]

nan_counts_poverty = df[poverty_columns].isna().sum().sort_values(ascending=False)
# nan_counts_suicide = df[suicide_columns].isna().sum().sort_values(ascending=False)
# nan_counts_completion = df[completion_columns].isna().sum().sort_values(ascending=False)
# nan_counts_unemployment = df[unemployment_columns].isna().sum().sort_values(ascending=False)


top_5_group_1 = nan_counts_poverty.head(5)
# top_5_group_2 = nan_counts_suicide.head(5)
# top_5_group_3 = nan_counts_completion.head(5)
# top_5_group_4 = nan_counts_unemployment.head(5)

print("Top 5 columns with the highest number of NaN values in Group 1:")
print(top_5_group_1)

# print("\nTop 5 columns with the highest number of NaN values in Group 2:")
# print(top_5_group_2)

# print("\nTop 5 columns with the highest number of NaN values in Group 3:")
# print(top_5_group_3)\

# print("\nTop 5 columns with the highest number of NaN values in Group 4:")
# print(top_5_group_4)

Top 5 columns with the highest number of NaN values in Group 1:
(value, 1.1.1 Population below international poverty line (%), Rural)          466
(value, 1.1.1 Population below international poverty line (%), Urban)          456
(value, 1.1.1 Population below international poverty line (%), 15-64 years)    447
(value, 1.1.1 Population below international poverty line (%), <15 years)      446
(value, 1.1.1 Population below international poverty line (%), 65+ years)      445
dtype: int64


In [25]:
poverty_rate_cols.shape

(744, 19)

In [22]:
poverty_rate_cols['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Ethiopia', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kuwait',
       'Kyrgyzstan', 'Laos', 'L

In [ ]:
countries_b = poverty_rate_cols[poverty_rate_cols['Country'].str.contains('Bangladesh')]
display(countries_b)